In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
import pywt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Function to load and preprocess images from multiple folders
def load_and_preprocess_images(folder_paths, label):
    images = []
    labels = []
    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
                image_path = os.path.join(folder_path, filename)
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (64, 64))  # Resize to a common size
                images.append(img)
                labels.append(label)
    return images, labels

# Define folder paths for pedestrian and non-pedestrian images
pedestrian_folders = [
    "/content/drive/MyDrive/ML1/JPEGImagesTest",
    "/content/drive/MyDrive/ML1/JPEGImagesTrain",
    # Add more pedestrian folders as needed
]

non_pedestrian_folders = [
    "/content/drive/MyDrive/ML1/JPEGImagesTest",
    # Add more non-pedestrian folders as needed
]

# Load and preprocess images from all pedestrian and non-pedestrian folders
pedestrian_images, pedestrian_labels = load_and_preprocess_images(pedestrian_folders, label=1)
non_pedestrian_images, non_pedestrian_labels = load_and_preprocess_images(non_pedestrian_folders, label=0)

# Combine pedestrian and non-pedestrian images and labels
X = np.concatenate([pedestrian_images, non_pedestrian_images], axis=0)
y = np.concatenate([pedestrian_labels, non_pedestrian_labels], axis=0)

# Extract HOG features from the combined images
def extract_hog_features(images):
    hog_features = []
    for img in images:
        # Calculate the number of cells and blocks based on the image size
        image_height, image_width = img.shape
        pixels_per_cell = (image_width // 8, image_height // 8)
        cells_per_block = (2, 2)

        # Extract HOG features
        features = hog(
            img,
            pixels_per_cell=pixels_per_cell,
            cells_per_block=cells_per_block,
            block_norm='L2-Hys',  # Use L2-Hys block normalization
            visualize=False,
            multichannel=False,  # Set to False for grayscale images
        )

        hog_features.append(features)

    return np.array(hog_features)

# Extract HOG features from the combined images
X_hog = extract_hog_features(X)

# Extract DWT features from the combined images
def extract_dwt_features(images):
    dwt_features = []
    for img in images:
        # Perform DWT on the image
        coeffs = pywt.dwt2(img, 'bior1.3')  # Use the Biorthogonal 1.3 wavelet
        LL, (LH, HL, HH) = coeffs  # Approximation and detail coefficients

        # Flatten and concatenate the coefficients
        features = np.concatenate((LL.ravel(), LH.ravel(), HL.ravel(), HH.ravel()))

        dwt_features.append(features)

    return np.array(dwt_features)

# Extract DWT features from the combined images
X_dwt = extract_dwt_features(X)

# Combine HOG and DWT features
X_combined = np.concatenate((X_hog, X_dwt), axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Create an SVM classifier and train it
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the model
accuracy = svm_model.score(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


<ipython-input-2-a3326c5a0b83>:54: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  features = hog(


Accuracy: 71.38%
